In [21]:
#data handling/prediction
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.externals.six import StringIO

from sklearn.cross_validation import KFold, train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn import metrics
import scipy.stats as stats

#visualization
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
import pydot
from sklearn.tree import export_graphviz

In [63]:
# Pull data in
loanData1 = pd.read_csv("/Users/ganderson/Dropbox/Developer/nyc16_ids1/data/lendingClub/LoanStats3a.csv")
loanData2 = pd.read_csv("/Users/ganderson/Dropbox/Developer/nyc16_ids1/data/lendingClub/LoanStats3b.csv")
loanData3 = pd.read_csv("/Users/ganderson/Dropbox/Developer/nyc16_ids1/data/lendingClub/LoanStats3c.csv")
loanData4 = pd.read_csv("/Users/ganderson/Dropbox/Developer/nyc16_ids1/data/lendingClub/LoanStats3d.csv")
loanData5 = pd.read_csv("/Users/ganderson/Dropbox/Developer/nyc16_ids1/data/lendingClub/LoanStats_2016Q1.csv")

In [87]:
# Combining data and inspecting it
tempFrame = pd.concat([loanData1,loanData2,loanData3,loanData4,loanData5], ignore_index=True)
# tempFrame.head(10) # Making sure concat worked
# tempFrame.shape # Looks correct
# tempFrame.dtypes # Need to clean up things like % inside int_rate and some other variables for data processing and drop off a ton of stuff.

In [88]:
# Dropping off features that aren't a pre-populated list
# Tried using np.logical to exclude a list of features based on the ones I wanted. Try troubleshooting later.
# scrubbedTempFrame = tempFrame.drop(np.logical_not(tempFrame[['int_rate','purpose','funded_amnt','verification_status','dti','earliest_cr_line','open_acc',
#                                                     'total_acc','revol_bal','revol_util','inq_last_6mths','mths_since_last_delinq','mths_since_last_record',
#                                                     'home_ownership','emp_length','addr_state','pub_rec','delinq_2yrs','term','loan_status','annual_inc',
#                                                     'collections_12_mths_ex_med','mths_since_last_major_derog','application_type','acc_now_delinq',
#                                                     'tot_cur_bal','out_prncp','total_pymnt','inq_last_12m','avg_cur_bal','mths_since_recent_bc_dlq',
#                                                     'mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_op_rev_tl',
#                                                     'num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m',
#                                                     'num_tl_op_past_12m','pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim',
#                                                     'total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit']], axis=1, inplace=True))
tempFrame.drop(tempFrame[['id','member_id','loan_amnt','funded_amnt_inv','installment','emp_title','issue_d',
                                     'pymnt_plan','url','desc','title','zip_code','initial_list_status','out_prncp_inv','total_pymnt_inv','total_rec_prncp',
                                     'total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','tot_coll_amt',
                                     'last_pymnt_amnt','next_pymnt_d','last_credit_pull_d','policy_code','annual_inc_joint','dti_joint',
                                     'verification_status_joint','open_acc_6m','open_il_6m','open_il_12m','open_il_24m','mths_since_rcnt_il',
                                     'total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim','inq_fi',
                                     'total_cu_tl','acc_open_past_24mths','bc_open_to_buy','bc_util','chargeoff_within_12_mths','delinq_amnt',
                                     'mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mort_acc',
                                     'mths_since_recent_bc','mths_since_recent_bc_dlq','num_actv_bc_tl','num_actv_rev_tl','num_bc_sats',
                                     'num_bc_tl','num_il_tl','acc_now_delinq','earliest_cr_line','sub_grade','application_type']], axis=1, inplace=True)

In [89]:
tempFrame.dtypes

funded_amnt                         int64
term                               object
int_rate                           object
grade                              object
emp_length                         object
home_ownership                     object
annual_inc                        float64
verification_status                object
loan_status                        object
purpose                            object
addr_state                         object
dti                               float64
delinq_2yrs                         int64
inq_last_6mths                    float64
mths_since_last_delinq            float64
mths_since_last_record            float64
open_acc                            int64
pub_rec                             int64
revol_bal                           int64
revol_util                         object
total_acc                           int64
out_prncp                         float64
total_pymnt                       float64
collections_12_mths_ex_med        

In [86]:
# Binarize term
tempFrame['term'] = [0 if x == " 36 months" else 1 for x in tempFrame['term']]
print tempFrame.dtypes.term # confirming this is now an int
print tempFrame.term.unique() # confirming this worked

# Convert int_rate into float and cut '%'
tempFrame['int_rate'] = tempFrame['int_rate'].str.replace('%', '')
tempFrame['int_rate'] = tempFrame['int_rate'].astype(float)
print tempFrame.dtypes.int_rate # confirming this worked

# OneHotEncoder grade

# Convert emp_length into int and cut '+' and 'years'
tempFrame['emp_length'] = tempFrame.str.replace('+ years','')
tempFrame['emp_length'] = tempFrame.str.replace('years','')
tempFrame['emp_length'] = tempFrame['emp_length'].astype(int)
print tempFrame.dtypes.emp_length

# OneHotEncoder home_ownership

# Convert revol_util to int and cut '%'
# OneHotEncoder verification_status
# loan status - THE THING I WANT TO PREDICT. Categorize using dummies?
# OneHotEncoder purpose
# OneHotEncoder addr_state

int64
[1]


AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas